In [12]:
import nltk
import re
import string

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
import pandas as pd
import numpy as np

In [14]:
data = pd.read_csv("/sentiment_tweets3.csv")
data.head()

,Index,message to examine,label (depression result)
0,106,just had a real good moment. i missssssssss hi...,0
1,217,is reading manga http://plurk.com/p/mzp1e,0
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,288,@lapcat Need to send 'em to my accountant tomo...,0
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0


In [15]:
data

,Index,message to examine,label (depression result)
0,106,just had a real good moment. i missssssssss hi...,0
1,217,is reading manga http://plurk.com/p/mzp1e,0
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,288,@lapcat Need to send 'em to my accountant tomo...,0
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0
...,...,...,...
10309,802309,No Depression by G Herbo is my mood from now o...,1
10310,802310,What do you do when depression succumbs the br...,1
10311,802311,Ketamine Nasal Spray Shows Promise Against Dep...,1
10312,802312,dont mistake a bad day with depression! everyo...,1


**Data Exploration**

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10314 entries, 0 to 10313
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Index                      10314 non-null  int64 
 1   message to examine         10314 non-null  object
 2   label (depression result)  10314 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 241.9+ KB


In [17]:
#check for null vals
data.isna().sum()

Index                        0
message to examine           0
label (depression result)    0
dtype: int64

In [18]:
data.drop(index=10313, inplace=True)

data

,Index,message to examine,label (depression result)
0,106,just had a real good moment. i missssssssss hi...,0
1,217,is reading manga http://plurk.com/p/mzp1e,0
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,288,@lapcat Need to send 'em to my accountant tomo...,0
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0
...,...,...,...
10308,802308,Many sufferers of depression aren't sad; they ...,1
10309,802309,No Depression by G Herbo is my mood from now o...,1
10310,802310,What do you do when depression succumbs the br...,1
10311,802311,Ketamine Nasal Spray Shows Promise Against Dep...,1


In [19]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Index,10313.0,491223.30864,261682.893213,106.0,263019.0,514969.0,773448.0,802312.0
label (depression result),10313.0,0.22428,0.417128,0.0,0.0,0.0,0.0,1.0


**Text preprocessing**

remove urls from the text

In [20]:
def remove_urls(text):
    url_pattern = re.compile(r'(https?://\S+)|(www\.\S+)|(\S+\.\S+/\S+)')
    return url_pattern.sub(r'', text)

data['urlsRemoved'] = data['message to examine'].apply(remove_urls)

data

,Index,message to examine,label (depression result),urlsRemoved
0,106,just had a real good moment. i missssssssss hi...,0,just had a real good moment. i missssssssss hi...
1,217,is reading manga http://plurk.com/p/mzp1e,0,is reading manga
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0,@comeagainjen -
3,288,@lapcat Need to send 'em to my accountant tomo...,0,@lapcat Need to send 'em to my accountant tomo...
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0,ADD ME ON MYSPACE!!!
...,...,...,...,...
10308,802308,Many sufferers of depression aren't sad; they ...,1,Many sufferers of depression aren't sad; they ...
10309,802309,No Depression by G Herbo is my mood from now o...,1,No Depression by G Herbo is my mood from now o...
10310,802310,What do you do when depression succumbs the br...,1,What do you do when depression succumbs the br...
10311,802311,Ketamine Nasal Spray Shows Promise Against Dep...,1,Ketamine Nasal Spray Shows Promise Against Dep...


In [22]:
#removing emoticons
def remove_emoji(text):
    emoji_pattern = re.compile("["
                                  u"\U0001F600-\U0001F64F"  # emoticons
                                  u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                  u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                  u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                  u"\U00002702-\U000027B0"  # other miscellaneous symbols
                                  u"\U000024C2-\U0001F251"  # enclosed characters
                                "]+", flags=re.UNICODE)

    return emoji_pattern.sub(r'', text)

data['emojiRemoved'] = data['urlsRemoved'].apply(remove_emoji)

data

,Index,message to examine,label (depression result),urlsRemoved,emojiRemoved
0,106,just had a real good moment. i missssssssss hi...,0,just had a real good moment. i missssssssss hi...,just had a real good moment. i missssssssss hi...
1,217,is reading manga http://plurk.com/p/mzp1e,0,is reading manga,is reading manga
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0,@comeagainjen -,@comeagainjen -
3,288,@lapcat Need to send 'em to my accountant tomo...,0,@lapcat Need to send 'em to my accountant tomo...,@lapcat Need to send 'em to my accountant tomo...
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0,ADD ME ON MYSPACE!!!,ADD ME ON MYSPACE!!!
...,...,...,...,...,...
10308,802308,Many sufferers of depression aren't sad; they ...,1,Many sufferers of depression aren't sad; they ...,Many sufferers of depression aren't sad; they ...
10309,802309,No Depression by G Herbo is my mood from now o...,1,No Depression by G Herbo is my mood from now o...,No Depression by G Herbo is my mood from now o...
10310,802310,What do you do when depression succumbs the br...,1,What do you do when depression succumbs the br...,What do you do when depression succumbs the br...
10311,802311,Ketamine Nasal Spray Shows Promise Against Dep...,1,Ketamine Nasal Spray Shows Promise Against Dep...,Ketamine Nasal Spray Shows Promise Against Dep...


In [23]:
#removing punctuations
import re

pattern = r"[^\w\s]"


data['punkt'] = data['emojiRemoved'].apply(lambda x: re.sub(pattern, " ", x))

data

,Index,message to examine,label (depression result),urlsRemoved,emojiRemoved,punkt
0,106,just had a real good moment. i missssssssss hi...,0,just had a real good moment. i missssssssss hi...,just had a real good moment. i missssssssss hi...,just had a real good moment i missssssssss hi...
1,217,is reading manga http://plurk.com/p/mzp1e,0,is reading manga,is reading manga,is reading manga
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0,@comeagainjen -,@comeagainjen -,comeagainjen
3,288,@lapcat Need to send 'em to my accountant tomo...,0,@lapcat Need to send 'em to my accountant tomo...,@lapcat Need to send 'em to my accountant tomo...,lapcat Need to send em to my accountant tomo...
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0,ADD ME ON MYSPACE!!!,ADD ME ON MYSPACE!!!,ADD ME ON MYSPACE
...,...,...,...,...,...,...
10308,802308,Many sufferers of depression aren't sad; they ...,1,Many sufferers of depression aren't sad; they ...,Many sufferers of depression aren't sad; they ...,Many sufferers of depression aren t sad they ...
10309,802309,No Depression by G Herbo is my mood from now o...,1,No Depression by G Herbo is my mood from now o...,No Depression by G Herbo is my mood from now o...,No Depression by G Herbo is my mood from now o...
10310,802310,What do you do when depression succumbs the br...,1,What do you do when depression succumbs the br...,What do you do when depression succumbs the br...,What do you do when depression succumbs the br...
10311,802311,Ketamine Nasal Spray Shows Promise Against Dep...,1,Ketamine Nasal Spray Shows Promise Against Dep...,Ketamine Nasal Spray Shows Promise Against Dep...,Ketamine Nasal Spray Shows Promise Against Dep...


In [27]:
from nltk.tokenize import word_tokenize
for i in data['message to examine']:
  var = word_tokenize(i)
  for j in var:
    print(j.lower())

Streaming output truncated to the last 5000 lines.
for
iso27001
?
our
client
has
received
confirmation
that
having
become
certified
they
'll
not
lose
their
â£500,000
contract
procastinated
by
doing
up
a
new
playlist
for
tcc
.
now
off
to
nalina
's
for
teh
18th
birthday
bash
.
2forever
working
on
simplifying
a
sales
strategy
and
offering
free
trials
of
www.marketdefender.com
interested
in
ppc
?
click
and
see
!
day
was
pretty
goood
actually
i
am
gon
na
be
pimping
tom
's
latest
podcast
show
later
on
tonight
as
well
-
it
's
srsly
fantastic
stuff
@
pkgulati
lol
i
'm
gna
be
there
too
,
that
's
why
i
was
asking
@
sidpiraya
wheres
the
fun
in
that
?
good
nite
everyone
sweet
dreams
@
piginthepoke
lol
i
see
,
it
's
just
that
i
have
met
8
people
in
the
last
2
years
who
were
there
,
one
of
whom
i
live
with
!
!
!
200gb
into
a
new
1tb
drive
...
@
fake_vyvyan
i
second
that
.
i
love
twitter
im
adicted
to
twitter
.
its
like
a
drug
watching
the
sound
of
music
happy
place
.
haha
i
decided
to
reply
to
peopl

KeyboardInterrupt: 

In [24]:
STOPWORDS = set(stopwords.words('english'))


def stopwords(text):
    return " ".join([word for word in text.split() if word not in STOPWORDS])

data['stopWord_Removed'] = data['punkt'].apply(stopwords)

data

,Index,message to examine,label (depression result),urlsRemoved,emojiRemoved,punkt,stopWord_Removed
0,106,just had a real good moment. i missssssssss hi...,0,just had a real good moment. i missssssssss hi...,just had a real good moment. i missssssssss hi...,just had a real good moment i missssssssss hi...,real good moment missssssssss much
1,217,is reading manga http://plurk.com/p/mzp1e,0,is reading manga,is reading manga,is reading manga,reading manga
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0,@comeagainjen -,@comeagainjen -,comeagainjen,comeagainjen
3,288,@lapcat Need to send 'em to my accountant tomo...,0,@lapcat Need to send 'em to my accountant tomo...,@lapcat Need to send 'em to my accountant tomo...,lapcat Need to send em to my accountant tomo...,lapcat Need send em accountant tomorrow Oddly ...
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0,ADD ME ON MYSPACE!!!,ADD ME ON MYSPACE!!!,ADD ME ON MYSPACE,ADD ME ON MYSPACE
...,...,...,...,...,...,...,...
10308,802308,Many sufferers of depression aren't sad; they ...,1,Many sufferers of depression aren't sad; they ...,Many sufferers of depression aren't sad; they ...,Many sufferers of depression aren t sad they ...,Many sufferers depression sad feel nothing per...
10309,802309,No Depression by G Herbo is my mood from now o...,1,No Depression by G Herbo is my mood from now o...,No Depression by G Herbo is my mood from now o...,No Depression by G Herbo is my mood from now o...,No Depression G Herbo mood done stressing peop...
10310,802310,What do you do when depression succumbs the br...,1,What do you do when depression succumbs the br...,What do you do when depression succumbs the br...,What do you do when depression succumbs the br...,What depression succumbs brain makes feel like...
10311,802311,Ketamine Nasal Spray Shows Promise Against Dep...,1,Ketamine Nasal Spray Shows Promise Against Dep...,Ketamine Nasal Spray Shows Promise Against Dep...,Ketamine Nasal Spray Shows Promise Against Dep...,Ketamine Nasal Spray Shows Promise Against Dep...


In [25]:
#Lemmatization
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def lemmatizeRows(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

data['lemmatizedRows'] = data['stopWord_Removed'].apply(lemmatizeRows)

data

,Index,message to examine,label (depression result),urlsRemoved,emojiRemoved,punkt,stopWord_Removed,lemmatizedRows
0,106,just had a real good moment. i missssssssss hi...,0,just had a real good moment. i missssssssss hi...,just had a real good moment. i missssssssss hi...,just had a real good moment i missssssssss hi...,real good moment missssssssss much,real good moment miss much
1,217,is reading manga http://plurk.com/p/mzp1e,0,is reading manga,is reading manga,is reading manga,reading manga,reading manga
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0,@comeagainjen -,@comeagainjen -,comeagainjen,comeagainjen,comeagainjen
3,288,@lapcat Need to send 'em to my accountant tomo...,0,@lapcat Need to send 'em to my accountant tomo...,@lapcat Need to send 'em to my accountant tomo...,lapcat Need to send em to my accountant tomo...,lapcat Need send em accountant tomorrow Oddly ...,lapcat Need send em accountant tomorrow Oddly ...
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0,ADD ME ON MYSPACE!!!,ADD ME ON MYSPACE!!!,ADD ME ON MYSPACE,ADD ME ON MYSPACE,ADD ME ON MYSPACE
...,...,...,...,...,...,...,...,...
10308,802308,Many sufferers of depression aren't sad; they ...,1,Many sufferers of depression aren't sad; they ...,Many sufferers of depression aren't sad; they ...,Many sufferers of depression aren t sad they ...,Many sufferers depression sad feel nothing per...,Many sufferer depression sad feel nothing pers...
10309,802309,No Depression by G Herbo is my mood from now o...,1,No Depression by G Herbo is my mood from now o...,No Depression by G Herbo is my mood from now o...,No Depression by G Herbo is my mood from now o...,No Depression G Herbo mood done stressing peop...,No Depression G Herbo mood done stressing peop...
10310,802310,What do you do when depression succumbs the br...,1,What do you do when depression succumbs the br...,What do you do when depression succumbs the br...,What do you do when depression succumbs the br...,What depression succumbs brain makes feel like...,What depression succumbs brain make feel like ...
10311,802311,Ketamine Nasal Spray Shows Promise Against Dep...,1,Ketamine Nasal Spray Shows Promise Against Dep...,Ketamine Nasal Spray Shows Promise Against Dep...,Ketamine Nasal Spray Shows Promise Against Dep...,Ketamine Nasal Spray Shows Promise Against Dep...,Ketamine Nasal Spray Shows Promise Against Dep...


In [26]:
#stemming
from nltk.stem import PorterStemmer
from nltk.stem import 	WordNetLemmatizer
stemmer = PorterStemmer()

def stemRows(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

data['stemmedRows'] = data['lemmatizedRows'].apply(stemRows)

data

,Index,message to examine,label (depression result),urlsRemoved,emojiRemoved,punkt,stopWord_Removed,lemmatizedRows,stemmedRows
0,106,just had a real good moment. i missssssssss hi...,0,just had a real good moment. i missssssssss hi...,just had a real good moment. i missssssssss hi...,just had a real good moment i missssssssss hi...,real good moment missssssssss much,real good moment miss much,real good moment miss much
1,217,is reading manga http://plurk.com/p/mzp1e,0,is reading manga,is reading manga,is reading manga,reading manga,reading manga,read manga
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0,@comeagainjen -,@comeagainjen -,comeagainjen,comeagainjen,comeagainjen,comeagainjen
3,288,@lapcat Need to send 'em to my accountant tomo...,0,@lapcat Need to send 'em to my accountant tomo...,@lapcat Need to send 'em to my accountant tomo...,lapcat Need to send em to my accountant tomo...,lapcat Need send em accountant tomorrow Oddly ...,lapcat Need send em accountant tomorrow Oddly ...,lapcat need send em account tomorrow oddli i e...
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0,ADD ME ON MYSPACE!!!,ADD ME ON MYSPACE!!!,ADD ME ON MYSPACE,ADD ME ON MYSPACE,ADD ME ON MYSPACE,add me on myspac
...,...,...,...,...,...,...,...,...,...
10308,802308,Many sufferers of depression aren't sad; they ...,1,Many sufferers of depression aren't sad; they ...,Many sufferers of depression aren't sad; they ...,Many sufferers of depression aren t sad they ...,Many sufferers depression sad feel nothing per...,Many sufferer depression sad feel nothing pers...,mani suffer depress sad feel noth persist nag ...
10309,802309,No Depression by G Herbo is my mood from now o...,1,No Depression by G Herbo is my mood from now o...,No Depression by G Herbo is my mood from now o...,No Depression by G Herbo is my mood from now o...,No Depression G Herbo mood done stressing peop...,No Depression G Herbo mood done stressing peop...,no depress g herbo mood done stress peopl deserv
10310,802310,What do you do when depression succumbs the br...,1,What do you do when depression succumbs the br...,What do you do when depression succumbs the br...,What do you do when depression succumbs the br...,What depression succumbs brain makes feel like...,What depression succumbs brain make feel like ...,what depress succumb brain make feel like neve...
10311,802311,Ketamine Nasal Spray Shows Promise Against Dep...,1,Ketamine Nasal Spray Shows Promise Against Dep...,Ketamine Nasal Spray Shows Promise Against Dep...,Ketamine Nasal Spray Shows Promise Against Dep...,Ketamine Nasal Spray Shows Promise Against Dep...,Ketamine Nasal Spray Shows Promise Against Dep...,ketamin nasal spray show promis against depres...


In [28]:
#clean tokens
def cleanData(text):

    ReapeatTokensRm =  " ".join( [ re.sub(r'(\w)\1{2,}', r'\1', word) for word in text.split()] )  # Remove repeating characters from tokens

    digitTokensRm =  " ".join( [ word for word in ReapeatTokensRm.split() if not re.search(r'\d', word) ] ) # Remove tokens containing digits

    underscoreTokensRm =  " ".join( [ word for word in digitTokensRm.split() if not re.search(r'_|\w*_\w*', word) ] ) # Remove tokens containing underscore

    specialTokensRm =  " ".join( [ word for word in underscoreTokensRm.split() if not re.search(r'[^a-zA-Z0-9\s]', word) ] ) # Remove tokens containing Special Characters

    return " ".join( [ word for word in specialTokensRm.split() if len(word) > 2 ] )  # Remove tokens less than 2 characters


data['cleanTokens'] = data['stemmedRows'].apply(cleanData)

data

,Index,message to examine,label (depression result),urlsRemoved,emojiRemoved,punkt,stopWord_Removed,lemmatizedRows,stemmedRows,cleanTokens
0,106,just had a real good moment. i missssssssss hi...,0,just had a real good moment. i missssssssss hi...,just had a real good moment. i missssssssss hi...,just had a real good moment i missssssssss hi...,real good moment missssssssss much,real good moment miss much,real good moment miss much,real good moment miss much
1,217,is reading manga http://plurk.com/p/mzp1e,0,is reading manga,is reading manga,is reading manga,reading manga,reading manga,read manga,read manga
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0,@comeagainjen -,@comeagainjen -,comeagainjen,comeagainjen,comeagainjen,comeagainjen,comeagainjen
3,288,@lapcat Need to send 'em to my accountant tomo...,0,@lapcat Need to send 'em to my accountant tomo...,@lapcat Need to send 'em to my accountant tomo...,lapcat Need to send em to my accountant tomo...,lapcat Need send em accountant tomorrow Oddly ...,lapcat Need send em accountant tomorrow Oddly ...,lapcat need send em account tomorrow oddli i e...,lapcat need send account tomorrow oddli even r...
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0,ADD ME ON MYSPACE!!!,ADD ME ON MYSPACE!!!,ADD ME ON MYSPACE,ADD ME ON MYSPACE,ADD ME ON MYSPACE,add me on myspac,add myspac
...,...,...,...,...,...,...,...,...,...,...
10308,802308,Many sufferers of depression aren't sad; they ...,1,Many sufferers of depression aren't sad; they ...,Many sufferers of depression aren't sad; they ...,Many sufferers of depression aren t sad they ...,Many sufferers depression sad feel nothing per...,Many sufferer depression sad feel nothing pers...,mani suffer depress sad feel noth persist nag ...,mani suffer depress sad feel noth persist nag ...
10309,802309,No Depression by G Herbo is my mood from now o...,1,No Depression by G Herbo is my mood from now o...,No Depression by G Herbo is my mood from now o...,No Depression by G Herbo is my mood from now o...,No Depression G Herbo mood done stressing peop...,No Depression G Herbo mood done stressing peop...,no depress g herbo mood done stress peopl deserv,depress herbo mood done stress peopl deserv
10310,802310,What do you do when depression succumbs the br...,1,What do you do when depression succumbs the br...,What do you do when depression succumbs the br...,What do you do when depression succumbs the br...,What depression succumbs brain makes feel like...,What depression succumbs brain make feel like ...,what depress succumb brain make feel like neve...,what depress succumb brain make feel like neve...
10311,802311,Ketamine Nasal Spray Shows Promise Against Dep...,1,Ketamine Nasal Spray Shows Promise Against Dep...,Ketamine Nasal Spray Shows Promise Against Dep...,Ketamine Nasal Spray Shows Promise Against Dep...,Ketamine Nasal Spray Shows Promise Against Dep...,Ketamine Nasal Spray Shows Promise Against Dep...,ketamin nasal spray show promis against depres...,ketamin nasal spray show promis against depres...


In [ ]:
#sentence segmentation
